In [1]:
import numpy as np
from osgeo import gdal
import os
import time as t

def write_geotiff(fname, data, geo_transform, projection):
    driver = gdal.GetDriverByName('GTiff')
    rows, cols = data.shape
    dataset = driver.Create(fname, cols, rows, 1, gdal.GDT_Int32)
    dataset.SetGeoTransform(geo_transform)
    dataset.SetProjection(projection)
    band = dataset.GetRasterBand(1)
    band.WriteArray(data)
    dataset = None  # Close the file

def load_data(data_name):
    data = str(data_name)
    dataset = gdal.Open(data, gdal.GA_ReadOnly)
    A = dataset.GetRasterBand(1)
    data=[]
    data.append(A.ReadAsArray())
    
    data = np.dstack(data)
    geo_transform = dataset.GetGeoTransform()
    proj = dataset.GetProjectionRef()
    del dataset, A
    return data, geo_transform, proj


def real_coord(image):

    image = gdal.Open(image, gdal.GA_ReadOnly)
    tl_x = top_left_x = image.GetGeoTransform()[0]
    tl_y = top_left_y = image.GetGeoTransform()[3]

    tr_x = top_right_x = image.GetGeoTransform()[0]
    tr_y = top_right_y = image.GetGeoTransform()[3] + image.RasterYSize*image.GetGeoTransform()[1]

    bl_x = bottom_left_x = image.GetGeoTransform()[0] + image.GetGeoTransform()[1] * image.RasterXSize
    bl_y = bottom_left_y = image.GetGeoTransform()[3]

    br_x = bottom_right_x = image.GetGeoTransform()[0] + image.GetGeoTransform()[1]*image.RasterXSize
    br_y = botoom_right_y = image.GetGeoTransform()[3] + image.RasterYSize*image.GetGeoTransform()[1]

    tl = (tl_x,tl_y)
    tr = (tr_x,tr_y)
    bl = (bl_x,bl_y)
    br = (br_x,br_y)
    print(tl)
    print(tr)
    print(bl)
    print(br)
    
    return tl, tr, bl, br


def real2array(value):
    return int(value/0.55000001+1)
    
def mosaic(img1, img2):
    zeros_mosaic = []
    tl_1, tr_1, bl_1, br_1 = real_coord(img1)
    tl_2, tr_2, bl_2, br_2 = real_coord(img2)

    tl_x1, tl_y1 = tl_1
    tr_x1, tr_y1 = tr_1
    bl_x1, bl_y1 = bl_1
    br_x1, br_y1 = br_1

    tl_x2, tl_y2 = tl_2
    tr_x2, tr_y2 = tr_2
    bl_x2, bl_y2 = bl_2
    br_x2, br_y2 = br_2
    

    if tl_x1 < tl_x2 and tl_y1 < tl_y2:
        
        x_absol = tl_x1
        y_absol = tl_y1
        x_size = real2array(max(br_x1, br_x2) - x_absol)
        y_size = real2array(max(br_y1, br_y2) - y_absol)
        zeros_mosaic = np.zeros((x_size,y_size))
        print("1")
        
        
        for i in range(real2array(tl_x1 - x_absol), real2array((br_x1 - x_absol))):
            for j in range(real2array(tl_y1 - y_absol), real2array((br_y1 - y_absol))):
                zeros_mosaic[i,j] = img1[i,j]
        
        for i in range(real2array(tl_x2 - x_absol), real2array(br_x2 - x_absol)):
            for j in range(real2array(tl_y2 - y_absol), real2array(br_y2 - y_absol)):
                zeros_mosaic[i,j] = img2[i,j]
        
        write_geotiff("Result.tif", zeros_mosaic, geo_transform1, proj)
        
        
    elif tl_x1 > tl_x2 and tl_y1 < tl_y2:
        x_absol = tl_x1
        y_absol = tl_y2
        x_size = real2array(max(br_x1, br_x2) - x_absol)
        y_size = real2array(max(br_y1, br_y2) - y_absol)
        print(x_size, y_size)
        print(2)
        zeros_mosaic = np.zeros((x_size,y_size))
        
        for i in range(real2array(tl_x1 - x_absol), real2array((br_x1 - x_absol))):
            for j in range(real2array(tl_y1 - y_absol), real2array((br_y1 - y_absol))):
                zeros_mosaic[i,j] = img1[i,j]
        
        for i in range(real2array(tl_x2 - x_absol), real2array(br_x2 - x_absol)):
            for j in range(real2array(tl_y2 - y_absol), real2array(br_y2 - y_absol)):
                zeros_mosaic[i,j] = img2[i,j]
                
        geo_transform1[0] = tl_x2
        geo_transform1[3] = tl_y1
        
        write_geotiff("Result.tif", zeros_mosaic, geo_transform1, proj)
        
    elif tl_x1 > tl_x2 and tl_y1 > tl_y2:
        x_absol = tl_x2
        y_absol = tl_y2
        img1_data, geo_transform1, proj = load_data(img1)
        img1 = gdal.Open(img1, gdal.GA_ReadOnly)
        img2_data, geo_transform2, proj = load_data(img2)
        img2 = gdal.Open(img2, gdal.GA_ReadOnly)
        x_size = real2array(max(br_x1, br_x2) - x_absol)
        y_size = real2array(max(br_y1, br_y2) - y_absol)
        
        
        A_start_col = real2array(tl_x1 - tl_x2)
        B_start_row = real2array(tl_y1 - tl_y2)
        
        zeros_mosaic = np.zeros((B_start_row+img2.RasterYSize,A_start_col+img1.RasterXSize))
        print(zeros_mosaic)
        
        for i in range(int(img1.RasterYSize)):
            for j in range(int(img1.RasterXSize)):
                zeros_mosaic[i,j + A_start_col] = img1_data[i,j]
        
        del img1_data
        
        #B_end_row = B_strat_row + img2.RasterYSize
        
        
        


        
        for i in range(int(img2.RasterYSize)):
            for j in range(int(img2.RasterXSize)):
                zeros_mosaic[i + B_start_row,j] = img2_data[i,j]
        
        bb = (tl_x2, 0.55000001, 0.0, tl_y2, 0.0, -0.55000001)
        
        
        write_geotiff("Result.tif", zeros_mosaic, bb, proj)
        

    elif tl_x1 < tl_x2 and tl_y1 > tl_y2:
        x_absol = tl_x2
        y_absol = tl_y1
        
        x_size = real2array(max(br_x1, br_x2) - x_absol)
        y_size = real2array(max(br_y1, br_y2) - y_absol)
        print("4")
        zeros_mosaic = np.zeros((x_size,y_size))
        
        for i in range(real2array(tl_x1 - x_absol), real2array((br_x1 - x_absol))):
            for j in range(real2array(tl_y1 - y_absol), real2array((br_y1 - y_absol))):
                zeros_mosaic[i,j] = img1[i,j]
        
        for i in range(real2array(tl_x2 - x_absol), real2array(br_x2 - x_absol)):
            for j in range(real2array(tl_y2 - y_absol), real2array(br_y2 - y_absol)):
                zeros_mosaic[i,j] = img2[i,j]
                
        geo_transform1[0] = tl_x1
        geo_transform1[3] = tl_y2
        
        write_geotiff("Result.tif", zeros_mosaic, geo_transform1, proj)
        
    
    


In [2]:
t1 = t.time()
img1 = ("E:/0_Image/Korea/겹침/1/K3A_20151030041620_03298_00044093_L1G_PS/K3A_20151030041620_03298_00044093_L1G_PG.TIF")
img2 = ("E:/0_Image/Korea/겹침/1/K3A_20160417041738_05867_00064869_L1G_PS/K3A_20160417041738_05867_00064869_L1G_PG.TIF")

zeros_mosaic = []
tl_1, tr_1, bl_1, br_1 = real_coord(img1)
tl_2, tr_2, bl_2, br_2 = real_coord(img2)

tl_x1, tl_y1 = tl_1
tr_x1, tr_y1 = tr_1
bl_x1, bl_y1 = bl_1
br_x1, br_y1 = br_1

tl_x2, tl_y2 = tl_2
tr_x2, tr_y2 = tr_2
bl_x2, bl_y2 = bl_2
br_x2, br_y2 = br_2

x_absol = tl_x2
y_absol = tl_y2
img1_data, geo_transform1, proj = load_data(img1)
img1 = gdal.Open(img1, gdal.GA_ReadOnly)
img2_data, geo_transform2, proj = load_data(img2)
img2 = gdal.Open(img2, gdal.GA_ReadOnly)
x_size = real2array(max(br_x1, br_x2) - x_absol)
y_size = real2array(max(br_y1, br_y2) - y_absol)


A_start_col = real2array(tl_x1 - tl_x2)
B_start_row = real2array(tl_y1 - tl_y2)

zeros_mosaic = np.zeros((B_start_row+img2.RasterYSize,A_start_col+img1.RasterXSize))
print(zeros_mosaic)

t_cal1 = t.time()
for i in range(int(img1.RasterYSize)):
    for j in range(int(img1.RasterXSize)):
        zeros_mosaic[i,j + A_start_col] = img1_data[i,j]

del img1_data

#B_end_row = B_strat_row + img2.RasterYSize

t_cal2 = t.time()




for i in range(int(img2.RasterYSize)):
    for j in range(int(img2.RasterXSize)):
        zeros_mosaic[i + B_start_row,j] = img2_data[i,j]
t_cal3 = t.time()
        
bb = (tl_x2, 0.55000001, 0.0, tl_y1, 0.0, -0.55000001)



write_geotiff("Result.tif", zeros_mosaic, bb, proj)
t2 = t.time()


(528098.38870486, 3942572.08188076)
(528098.38870486, 3959642.98219114)
(548975.28908444, 3942572.08188076)
(548975.28908444, 3959642.98219114)
(511698.29031365, 3940073.50289957)
(511698.29031365, 3957210.40321115)
(532861.19069843, 3940073.50289957)
(532861.19069843, 3957210.40321115)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [6]:
print(t2-t1)
print(-(t_cal1 - t_cal2))
print(-(t_cal1 - t_cal3))


1090.9277381896973
521.0942177772522
1063.1096954345703
